In [1]:
# --- Bevezetés: Könyvtárak importálása
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

In [ ]:
# --- 1. Lépés: Embeddingek betöltése

# Fájl elérési út (módosítsd saját elérésed szerint)
embedding_file = '/Users/zelenyianszkimate/Documents/Szakdolgozat/processed_data/processed_documents_with_embeddings.parquet'

# Betöltés
embedding_df = pd.read_parquet(embedding_file)
print(f"Betöltött dokumentumok száma: {len(embedding_df)}")

# --- 2. Lépés: Embeddingek alapellenőrzése

# Ellenőrizzük az embeddingek típusát és dimenzióját
print(f"Első embedding típusa: {type(embedding_df['embedding'].iloc[0])}")
print(f"Első embedding dimenziója: {len(embedding_df['embedding'].iloc[0])}")

# Hiányzó embeddingek számának ellenőrzése
missing_embeddings = embedding_df['embedding'].isna().sum()
print(f"Hiányzó embeddingek száma: {missing_embeddings}")

# --- 3. Lépés: PCA dimenziócsökkentés

# Embeddingek NumPy tömbbé alakítása
# Először távolítsuk el a hiányzó embeddingeket tartalmazó sorokat a DataFrame-ből
valid_embeddings_df = embedding_df.dropna(subset=['embedding'])
# Majd alakítsuk a Series-t listává a vstack számára
X = np.vstack(valid_embeddings_df['embedding'].tolist())


# PCA futtatása
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

print(f"PCA első két komponens magyázott varianciája: {pca.explained_variance_ratio_}")

# --- 4. Lépés: PCA eredmény vizualizáció

plt.figure(figsize=(10,7))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], s=2, alpha=0.6)
plt.title("Embeddingek PCA 2D leképezése")
plt.xlabel("Főkomponens 1")
# --- 5. Lépés (opcionális): Jogterületi színezés, ha van ilyen mező

if 'JogTerulet' in embedding_df.columns:
    # Használjuk a már szűrt DataFrame-et, hogy a címkék száma megegyezzen a redukált pontok számával
    jogterulet_labels = valid_embeddings_df['JogTerulet'].fillna('ismeretlen').values

    plt.figure(figsize=(12,8))
    sns.scatterplot(x=X_reduced[:,0], y=X_reduced[:,1], hue=jogterulet_labels, s=8, palette='tab10')
    plt.title("Embeddingek jogterület szerint színezve")
    plt.xlabel("Főkomponens 1")
    plt.ylabel("Főkomponens 2")
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.show()
else:
    print("Jogterület mező nem található a DataFrame-ben.")

# --- Vége: Kiértékelés

print("EDA lépések befejezve. Ellenőrizd az ábrákat és a szóródást.")
